In [ ]:
using Revise

using JDF

using CSV, DataFrames, Blosc

# use 12 threads
Blosc.set_num_threads(6)

@time a = CSV.read("C:/data/Performance_All/Performance_2010Q3.txt", delim = '|', header = false);

In [2]:
head(a) = first(a, 10)
head(a)

UndefVarError: UndefVarError: a not defined

In [ ]:
using TableView
showtable(a)

## Save column

In [ ]:
@time b = Array(a[!, :Column5])
io = iow()
@time metadata = compress_then_write(b, io)
close(io)

In [ ]:
metadata

## Load column

In [ ]:
using JDF
using CSV, DataFrames
io = ior()
@time oo = column_loader(eltype(b), io, metadata)
close(io)

## Compare results

In [ ]:
all(skipmissing(b) .== skipmissing(oo))

## Save a DataFrame

In [ ]:
@time metadatas = savejdf(a, "c:/data/a.jdf");
using JLSO
JLSO.save("c:/data/a.metadata.jlso", metadatas)

In [ ]:
# using JLSO

# @time JLSO.save("c:/data/a.jlso", a)

In [ ]:
# using Feather
# @time Feather.write("c:/data/a.feather", a)

In [ ]:
# using JLD2
# @time JLD2.@save "c:/data/a.jld2" a

In [ ]:
# loadfromjld2 = @time JLD2.@load "c:/data/a.jld2"

In [ ]:
filesize("c:/data/a.jdf")/1024^3

## Load a DataFrame

In [3]:
using JLSO, JDF
metadatas = JLSO.load("c:/data/a.metadata.jlso")["data"]

┌ Info: Precompiling JDF [babc3d20-cd49-4f60-a736-a8f9c08892d3]
└ @ Base loading.jl:1264


(names = Symbol[:Column1, :Column2, :Column3, :Column4, :Column5, :Column6, :Column7, :Column8, :Column9, :Column10  …  :Column22, :Column23, :Column24, :Column25, :Column26, :Column27, :Column28, :Column29, :Column30, :Column31], rows = 23954669, metadatas = Any[(len = 8097949, type = Int64), (string_compressed_bytes = 239546690, string_len_bytes = 790966, type = String), (Tmeta = (string_compressed_bytes = 9731855, string_len_bytes = 3536508, type = String), missingmeta = (len = 2486476, type = Bool), type = Union{Missing, String}), (len = 4362428, type = Float64), (Tmeta = (len = 155212867, type = Float64), missingmeta = (len = 3349566, type = Bool), type = Union{Missing, Float64}), (len = 6056606, type = Int64), (len = 5239061, type = Int64), (Tmeta = (len = 13442895, type = Int64), missingmeta = (len = 116523, type = Bool), type = Union{Missing, Int64}), (string_compressed_bytes = 167682683, string_len_bytes = 790966, type = String), (len = 5214798, type = Int64)  …  (Tmeta = (len

In [ ]:
@time a2 = loadjdf("c:/data/a.jdf", metadatas);

## Compare

In [ ]:
all(names(a) .== names(a2))
all(skipmissing([all(a2[!,name] .== Array(a[!,name])) for name in names(a2)]))

In [2]:
2+2

4

In [5]:
metadata = metadatas.metadatas

31-element Array{Any,1}:
 (len = 8097949, type = Int64)                                                                                                                                       
 (string_compressed_bytes = 239546690, string_len_bytes = 790966, type = String)                                                                                     
 (Tmeta = (string_compressed_bytes = 9731855, string_len_bytes = 3536508, type = String), missingmeta = (len = 2486476, type = Bool), type = Union{Missing, String}) 
 (len = 4362428, type = Float64)                                                                                                                                     
 (Tmeta = (len = 155212867, type = Float64), missingmeta = (len = 3349566, type = Bool), type = Union{Missing, Float64})                                             
 (len = 6056606, type = Int64)                                                                                                                   

get_bytes (generic function with 1 method)

In [26]:
maximum(get_bytes.(metadata))

239546690

In [23]:
for (i, m) in enumerate(metadata)
    get_bytes(m)
end